# Trains and Tests PLDA

In [155]:
#!/usr/bin/env python3.7
import os
import sys
sys.path.insert(0, "/export/c10/lmorove1/PythonLibs/plda")
import plda
import numpy as np
import matplotlib.pyplot as plt
import kaldi_io
import argparse
import pickle
from sklearn.decomposition import PCA

import pandas as pd 

from sklearn.neighbors import NearestNeighbors


#PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import decomposition

In [118]:
DATA_PATH = '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/'

!pwd && ls
os.chdir(DATA_PATH)
!pwd && ls

/export/c08/lmorove1/kaldi/egs/beatPDivec/v1
cmd.sh	 conf  exp    mfcc     README.txt  run.sh   sid    utils
cmd.sh~  data  local  path.sh  runFor.sh   run.sh~  steps
/export/c08/lmorove1/kaldi/egs/beatPDivec/v1
cmd.sh	 conf  exp    mfcc     README.txt  run.sh   sid    utils
cmd.sh~  data  local  path.sh  runFor.sh   run.sh~  steps


In [119]:
# def be_pca_pldax(sFileTrai, sFileTest, sOut, iComponents):
# # os.chdir("/export/c10/lmorove1/kaldi/egs/xVecPD/v2_NeuroConc16k")
# # sFileTrai="/export/c10/lmorove1/kaldi/egs/xVecPD/v2_NeuroConc16k/exp/sre18nn/xvectors_Training_Fold1/xvector.scp"
# # sOut = Folder where to store results 
# # iComponents = nb of classes (nb of measurements_id? or nb of subject_ids?)
        
#         dIvecTrai = { key:mat for key,mat in kaldi_io.read_vec_flt_scp(sFileTrai) }
#         dTraiPD= dict()
#         dTraiHC= dict()
#         for key, value in dIvecTrai.items():
#             print(key)
#             if '1007' in str(key):
#                 dTraiPD[key] = value
#             else:
#                 dTraiHC[key]=value
        
#         print('len(dTraiPD) : ', len(dTraiPD))
#         print('len(dTraiHC) : ', len(dTraiHC))
#         vTrai=np.concatenate((np.array(list(dTraiPD.values()), dtype=float), np.array(list(dTraiHC.values()), dtype=float)))  
#         vLTrai=np.concatenate((np.zeros(len(dTraiPD)), np.ones(len(dTraiHC))))

# # sFileTest="/export/c10/lmorove1/kaldi/egs/xVecPD/v2_NeuroConc16k/exp/sre18nn/xvectors_Test_Fold1/xvector.scp"

        
#         dIvecTest = { key:mat for key,mat in kaldi_io.read_vec_flt_scp(sFileTest) }
#         dTestPD= dict()
#         dTestHC= dict()

#         for key, value in dIvecTest.items():
#             if 'PD' in str(key):
#                 dTestPD[key] = value
#             else:
#                 dTestHC[key]=value

#         print('len(dTestPD) : ', len(dTestPD))
#         print('len(dTestHC) : ', len(dTestHC))
#         vTest=np.concatenate((np.array(list(dTestPD.values()), dtype=float), np.array(list(dTestHC.values()), dtype=float)))
#         vLTest=np.concatenate((np.zeros(len(dTestPD)), np.ones(len(dTestHC))))
        
#         #iComponents=60;

#         if isinstance(iComponents, str):
#                 iComponents=int(iComponents)

                
#         pca = PCA(n_components=iComponents, svd_solver='randomized',
#                             whiten=True).fit(vTrai)

#         vTraiPCA=pca.transform(vTrai)
#         vTestPCA=pca.transform(vTest)
        
# #         classifier = plda.Classifier()
# #         classifier.fit_model(vTraiPCA, vLTrai)

# #         predictions, log_p_predictions = classifier.predict(vTestPCA)
# #         predictionsTrai, log_p_predictionsTrai = classifier.predict(vTraiPCA)
        
# #         print('Accuracy: {}'.format((vLTest == predictions).mean()))
# #         print('NumObs: {}'.format((len(vLTest))))
# #         print('PCAComponents: {}'.format((iComponents)))
# #         print('iNumPDTrai: {}'.format((len(dTraiPD))))
# #         print('iNumHCTrai: {}'.format((len(dTraiHC))))
# #         print('ScoresTrai:')
# #         print(log_p_predictionsTrai)
# #         print('#')
# #         print('#')
# #         print('iNumPDTest: {}'.format((len(dTestPD))))
# #         print('iNumHCTest: {}'.format((len(dTestHC))))
# #         print('ScoresTest:')
# #         print(log_p_predictions)
        
# #         if not  os.path.isdir(sOut):
# #                 os.mkdir(sOut)
        
# #         with open(os.path.join(sOut,'objs.pkl'), 'wb') as f:  # Python 3: open(..., 'wb')
# #             pickle.dump([predictions,log_p_predictions,vLTest], f)


In [311]:
def be_pca_pldax(sFileTrai, sFileTest, sOut, iComponents):
# os.chdir("/export/c10/lmorove1/kaldi/egs/xVecPD/v2_NeuroConc16k")
# sFileTrai="/export/c10/lmorove1/kaldi/egs/xVecPD/v2_NeuroConc16k/exp/sre18nn/xvectors_Training_Fold1/xvector.scp"
# sOut = Folder where to store results 
# iComponents = nb of classes (nb of measurements_id? or nb of subject_ids?)
    dIvecTrai = { key:mat for key,mat in kaldi_io.read_vec_flt_scp(sFileTrai) }
    vTrai= pd.DataFrame((list(dIvecTrai.values())))
    vLTrai = np.array([x[-1] for x in np.array(list(dIvecTrai.keys()))])
    
    # FIXME : For realPD, we need more than -5 (CIS-PD subject_id is 4 characters long)
    # FIXME REAL-PD it's not only int 
    vTraiSubjectId = np.array(([int(x[-5:-1]) for x in np.array(list(dIvecTrai.keys()))]))

    dIvecTest = { key:mat for key,mat in kaldi_io.read_vec_flt_scp(sFileTest) }
    vTest=np.array(list(dIvecTest.values()), dtype=float)
    vLTest=np.array([x[-1] for x in np.array(list(dIvecTest.keys()))])
    vTestSubjectId = np.array([int(x[-5:-1]) for x in np.array(list(dIvecTest.keys()))])
    
    
    # I don't know why i need to reshape. is it normal that I have a 1D array?
    print('before transform : ', vTrai.shape)
    print('before transform : ', vTest.shape)
    #vTrai = vTrai.reshape(-1, 1)
    #vTest = vTrai.reshape(-1, 1)
    #iComponents=60;

    if isinstance(iComponents, str):
            iComponents=int(iComponents)


    pca = PCA(n_components=iComponents, svd_solver='randomized', whiten=True)
    pca.fit(vTrai)


    vTraiPCA=pca.transform(vTrai)
    vTestPCA=pca.transform(vTest)

    print(vTraiPCA.shape)
    print(vTestPCA.shape)
    return vTraiPCA, vLTrai, vTraiSubjectId, vTestPCA, vLTest, vTestSubjectId
#         classifier = plda.Classifier()
#         classifier.fit_model(vTraiPCA, vLTrai)

#         predictions, log_p_predictions = classifier.predict(vTestPCA)
#         predictionsTrai, log_p_predictionsTrai = classifier.predict(vTraiPCA)
        
#         print('Accuracy: {}'.format((vLTest == predictions).mean()))
#         print('NumObs: {}'.format((len(vLTest))))
#         print('PCAComponents: {}'.format((iComponents)))
#         print('iNumPDTrai: {}'.format((len(dTraiPD))))
#         print('iNumHCTrai: {}'.format((len(dTraiHC))))
#         print('ScoresTrai:')
#         print(log_p_predictionsTrai)
#         print('#')
#         print('#')
#         print('iNumPDTest: {}'.format((len(dTestPD))))
#         print('iNumHCTest: {}'.format((len(dTestHC))))
#         print('ScoresTest:')
#         print(log_p_predictions)
        
#         if not  os.path.isdir(sOut):
#                 os.mkdir(sOut)
        
#         with open(os.path.join(sOut,'objs.pkl'), 'wb') as f:  # Python 3: open(..., 'wb')
#             pickle.dump([predictions,log_p_predictions,vLTest], f)


In [344]:
sFileTrai = '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/exp/ivectors_Training_Fold0/ivector.scp'
sFileTest = '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/exp/ivectors_Testing_Fold0/ivector.scp'
sOut = '' #None because it's only needed for PLDA 
iComponents = 5 # FIXME : should be 5 ????? from 0 to 4 for on/off

vTraiPCA, vLTrai, vTraiSubjectId, vTestPCA, vLTest, vTestSubjectId = be_pca_pldax(sFileTrai, sFileTest, sOut, iComponents)

before transform :  (1403, 300)
before transform :  (364, 300)
(1403, 5)
(364, 5)


In [227]:
print(type(vTraiSubjectId[0]))

<class 'numpy.int64'>


In [313]:
a=np.array([1,2,3,1,2,3,1,2,3])
b=np.array([1])
c = np.where(a == b)
d = np.array([10,11,12,13,14,15,16,17,18,19])
print(c)

print(d[c])

# print(c[0])
# print(d)
# print(np.isin(d, c[0]))
# d = d[np.isin(d, c[0])]
# print(d)
# d[np.isin(d[:,0],m)]

(array([0, 3, 6]),)
[10 13 16]


In [257]:
c

(array([0, 3, 6]),)

In [248]:
indices_subject_id

(array([], dtype=int64),)

In [308]:
for subject_id in np.unique(vTraiSubjectId):
    print(subject_id)

1004
1006
1007
1019
1020
1023
1032
1034
1038
1039
1043
1044
1048
1049
1051


In [309]:
for subject_id in np.unique(vTestSubjectId):
    print(subject_id)

1004
1006
1007
1019
1020
1023
1032
1034
1038
1039
1043
1044
1048
1049
1051


In [346]:
# Divide vTrainPCA by patient 

#from sklearn.neighbors import NeighborhoodComponentsAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# KNN 
# AND MEANs cosine distance 


for subject_id in np.unique(vTraiSubjectId):
    print('----- ' + str(subject_id) + '----- ')
    knn = KNeighborsClassifier(n_neighbors=3)
    
    # Filter vTraiPCA and vLTraiPCA for one subject_id
    indices_subject_id = np.where(vTraiSubjectId == subject_id) # HAPPY
#     print('indices_subject_id : ', indices_subject_id)
#     print(type(vTraiPCA))
#     print(type(vLTrai))
    vTraiPCA_subjectid = vTraiPCA[indices_subject_id]
    vLTrai_subjectid = vLTrai[indices_subject_id]
#     print('len(vTraiPCA_subjectid) : ', len(vTraiPCA_subjectid))
#     print('len(vLTrai_subjectid) : ', len(vLTrai_subjectid))
    
    # Filter vTestPCA and vLTestPCA for one subject_id
    indices_subject_id = np.where(vTestSubjectId == subject_id)
#     print('TEST indices_subject_id : ', indices_subject_id)
    vTestPCA_subjectid = vTestPCA[indices_subject_id]
    vLTest_subjectid = vLTest[indices_subject_id]
#     print('len(vTestPCA_subjectid) : ', len(vTestPCA_subjectid))
#     print('len(vLTest_subjectid) : ', len(vLTest_subjectid))

    knn.fit(vTraiPCA_subjectid, vLTrai_subjectid)
    
    print('Training KNN Train Scores')
    print(knn.score(vTraiPCA_subjectid, vLTrai_subjectid))
    print('Testing KNN Train Scores')
    print(knn.score(vTestPCA_subjectid, vLTest_subjectid))

#vTraiPCA, vLTrai, 

----- 1004----- 
Training KNN Train Scores
0.5230769230769231
Testing KNN Train Scores
0.11764705882352941
----- 1006----- 
Training KNN Train Scores
0.7931034482758621
Testing KNN Train Scores
0.625
----- 1007----- 
Training KNN Train Scores
0.6388888888888888
Testing KNN Train Scores
0.48333333333333334
----- 1019----- 
Training KNN Train Scores
0.6666666666666666
Testing KNN Train Scores
0.0
----- 1020----- 
Training KNN Train Scores
0.8012820512820513
Testing KNN Train Scores
0.9743589743589743
----- 1023----- 
Training KNN Train Scores
0.5
Testing KNN Train Scores
0.2727272727272727
----- 1032----- 
Training KNN Train Scores
0.6170212765957447
Testing KNN Train Scores
0.2777777777777778
----- 1034----- 
Training KNN Train Scores
0.6875
Testing KNN Train Scores
0.375
----- 1038----- 
Training KNN Train Scores
0.7333333333333333
Testing KNN Train Scores
0.7142857142857143
----- 1039----- 
Training KNN Train Scores
0.5961538461538461
Testing KNN Train Scores
0.34615384615384615
-----

In [333]:
arr = np.array(['11', '12', '13', '14', '15', '16', '17', '15', '11', '12', '14', '15', '16', '17'])

result = np.where(arr == '15')
 
print('Tuple of arrays returned : ', result)
print("Elements with value 15 exists at following indices", result[0], sep='\n')

Tuple of arrays returned :  (array([ 4,  7, 11]),)
Elements with value 15 exists at following indices
[ 4  7 11]


In [ ]:
#from sklearn.neighbors import NeighborhoodComponentsAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# KNN 
# AND MEANs cosine distance 
knn = KNeighborsClassifier(n_neighbors=3)

#nca = NeighborhoodComponentsAnalysis(random_state=42)

#nca_pipe = Pipeline([('nca', nca), ('knn', knn)])


knn.fit(vTraiPCA, vLTrai)

print(knn.score(vTraiPCA, vLTrai))
print(knn.score(vTestPCA, vLTest))

# nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(vTraiPCA)
# distances, indices = nbrs.kneighbors(vTraiPCA)
# indices

In [102]:
knn = KNeighborsClassifier(n_neighbors=2)

#nca = NeighborhoodComponentsAnalysis(random_state=42)

#nca_pipe = Pipeline([('nca', nca), ('knn', knn)])

knn.fit(vTraiPCA, vLTrai)

print(knn.score(vTraiPCA, vLTrai))
print(knn.score(vTestPCA, vLTest))

0.6436208125445474
0.41208791208791207


In [ ]:
>>> from sklearn.neighbors import (NeighborhoodComponentsAnalysis,
... KNeighborsClassifier)
>>> from sklearn.datasets import load_iris
>>> from sklearn.model_selection import train_test_split
>>> from sklearn.pipeline import Pipeline
>>> X, y = load_iris(return_X_y=True)
>>> X_train, X_test, y_train, y_test = train_test_split(X, y,
... stratify=y, test_size=0.7, random_state=42)
>>> nca = NeighborhoodComponentsAnalysis(random_state=42)
>>> knn = KNeighborsClassifier(n_neighbors=3)
>>> nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
>>> nca_pipe.fit(X_train, y_train)
Pipeline(...)
>>> print(nca_pipe.score(X_test, y_test))
0.96190476...